In [1]:
import trimesh
def pytorch3d_to_trimesh(p3d_mesh):
    # Get vertices and faces from the PyTorch3D mesh
    vertices = p3d_mesh.verts_packed().detach().cpu().numpy()
    faces = p3d_mesh.faces_packed().detach().cpu().numpy()

    # Create a Trimesh object
    return trimesh.Trimesh(vertices=vertices, faces=faces)

In [ ]:
# If you want to have meshes with different number of vertices:




In [ ]:
def equality_constraints(self, xs: torch.Tensor, y: torch.Tensor):
    """
    Enforces volume constraint
    Assumes same number of vertices in each projected mesh currently

    Args:
        xs (tensor): vertices of original mesh, B x V x 3
        y (tensor): vertices of projected mesh, B x V x 3
    """
    n_batches = len(xs)
    verts = y.view(n_batches,-1,3) # (B, V, 3)

    faces = self.src.faces_padded().view(n_batches, -1, 3).detach()  # (B, max F_i, 3)
    _, n_faces, _ = faces.size()
    batch_indices = torch.arange(n_batches)[:, None, None].expand(-1, n_faces, 3).detach() # detached

    face_vertices = verts[batch_indices, faces]  # (B, F, 3, 3)
    # Calculate volume
    v0, v1, v2 = face_vertices[:, :, 0, :], face_vertices[:, :, 1, :], face_vertices[:, :, 2, :]
    cross = torch.cross(v1, v2, dim=-1)  # (B, F, 3)
    face_volumes = torch.einsum('bfi,bfi->bf', v0, cross) / 6.0  # (B, F)

    volumes = torch.sum(face_volumes, dim=1).abs() # (B,)
    return volumes  # (B,)
